In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [5]:
# TODO: FILL IN PARAMETERS
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "tinyimagenet"
dataset_short = "tinyimage"
DATASET_ENUM = DatasetClassN.Tinyimage

## Attack model training parameters
LEARNING_R = 0.001
EPOCHS = 3
MULTI_WORKERS_N = 4

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [6]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [7]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [8]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 25000
Batchsize: 1
Inputs:tensor([[[[0.1686, 0.0980, 0.0431,  ..., 0.1608, 0.4510, 0.3725], [0.2078, 0.1412, 0.0745,  ..., 0.3412, 0.3961, 0.3922], [0.0745, 0.0588, 0.0314,  ..., 0.9647, 0.8078, 0.3451], ..., [0.7373, 0.7529, 0.7647,  ..., 0.1804, 0.4353, 0.2667], [0.7843, 0.7804, 0.7647,  ..., 0.4118, 0.2078, 0.1765], [0.7373, 0.8196, 0.8353,  ..., 0.2510, 0.2196, 0.3882]],[[0.2196, 0.1490, 0.0941,  ..., 0.1490, 0.4275, 0.3412], [0.2588, 0.1922, 0.1255,  ..., 0.3373, 0.3765, 0.3608], [0.1176, 0.1020, 0.0745,  ..., 0.9647, 0.7922, 0.3294], ..., [0.7412, 0.7490, 0.7608,  ..., 0.3490, 0.6039, 0.4353], [0.7765, 0.7725, 0.7529,  ..., 0.5804, 0.3765, 0.3451], [0.7294, 0.8118, 0.8118,  ..., 0.4196, 0.3882, 0.5569]],[[0.1412, 0.0706, 0.0235,  ..., 0.0000, 0.2863, 0.2000], [0.1882, 0.1216, 0.0549,  ..., 0.1647, 0.2196, 0.2078], [0.0549, 0.0392, 0.0118,  ..., 0.7686, 0.6039, 0.1451], ..., [0

In [9]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 2.5064,  1.4358,  0.8869]],  [[ 1.8374,  1.4284,  1.4190]],  [[ 1.5731,  1.0329,  0.8880]],  [[ 1.3434,  0.3086,  0.2857]],  [[ 1.9014,  1.3780,  1.0226]],  [[ 1.2728,  1.1621,  0.4431]],  [[ 3.6064,  3.0829,  2.4454]],  [[ 2.0238,  1.8456,  1.3820]],  [[ 3.9507,  3.5827,  3.5707]],  [[ 0.6999,  0.4235, -0.2055]],  [[ 1.8878,  1.3992,  1.2878]],  [[ 1.7385,  1.2451,  0.9985]],  [[ 1.4069,  0.9085,  0.8559]],  [[ 0.8502,  0.7263,  0.3458]],  [[ 4.5617,  3.6527,  3.5601]],  [[ 1.5597,  1.3889,  1.3649]],  [[ 1.5634,  1.3611,  1.3303]],  [[ 1.9588,  1.5697,  1.5209]],  [[ 4.6112,  4.0604,  3.6159]],  [[ 0.8242,  0.7582,  0.4912]],  [[ 2.3736,  2.1479,  1.6192]],  [[ 1.9770,  1.8410,  1.5104]],  [[ 0.1689, -0.1468, -0.2264]],  [[ 1.0595,  0.7415,  0.5205]],  [[ 1.9669,  1.6317,  0.8175]],  [[ 2.6594,  2.1397,  0.8045]],  [[ 1.1811,  1.0765,  1.0655]],  [[ 2.4324,  1.6373,  0.3616]],  [[ 2.2366,  1.5583,  1.2347]],  [[ 3.6124,  0.9714,  0.6698]],  [[ 1.4101,  1.0816,  1.0221]], 

### Attack Model

Simple Model

In [10]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [11]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
       

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [12]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [13]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DEVICE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.9686, 0.9686, 0.9725,  ..., 0.9765, 0.9725, 0.9686], [0.9608, 0.9647, 0.9647,  ..., 0.9725, 0.9686, 0.9686], [0.9647, 0.9647, 0.9686,  ..., 0.9686, 0.9647, 0.9647], ..., [0.3843, 0.1294, 0.3294,  ..., 0.6627, 0.6980, 0.4980], [0.1333, 0.3804, 0.4118,  ..., 0.4078, 0.5020, 0.2902], [0.1569, 0.2863, 0.4353,  ..., 0.4510, 0.6549, 0.4078]],[[0.9765, 0.9765, 0.9804,  ..., 0.9843, 0.9804, 0.9765], [0.9686, 0.9725, 0.9725,  ..., 0.9804, 0.9765, 0.9765], [0.9725, 0.9725, 0.9765,  ..., 0.9765, 0.9725, 0.9725], ..., [0.3647, 0.1137, 0.3176,  ..., 0.5765, 0.6118, 0.4118], [0.1098, 0.3608, 0.4000,  ..., 0.3098, 0.4039, 0.1922], [0.1294, 0.2667, 0.4235,  ..., 0.3490, 0.5490, 0.3020]],[[0.9333, 0.9333, 0.9373,  ..., 0.9412, 0.9373, 0.9333], [0.9255, 0.9294, 0.9294,  ..., 0.9373, 0.9333, 0.9333], [0.9294, 0.9294, 0.9333,  ..., 0.9333, 0.9294, 0.9294], ..., [0.1451, 0.0000,

Attack Evaluation DL Sample

In [14]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

### Train Attack model

In [ ]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 30000
Batchsize: 64
Inputs:tensor([[[ 1.5482e+01, -1.1628e+00, -1.8282e+00]],  [[ 1.1281e+01,  3.8217e+00, -1.9052e+00]],  [[ 1.2290e+01,  9.8166e-01, -1.2734e+00]],  [[ 6.6359e+00,  4.1631e-01,  2.3707e-01]],  [[ 3.7553e+00,  1.1720e+00,  5.5990e-01]],  [[ 1.1729e+01, -4.6245e-01, -7.9813e-01]],  [[ 7.5011e+00,  1.5680e+00,  8.9420e-01]],  [[ 6.1661e+00,  1.0230e+00, -3.3113e-01]],  [[ 8.7063e+00,  6.9468e+00, -2.2595e+00]],  [[ 3.4986e+00,  1.7438e+00,  5.7986e-02]],  [[ 2.4196e+00,  1.7815e+00,  1.6315e+00]],  [[ 4.3901e+00,  1.9657e+00,  4.9074e-01]],  [[ 9.1401e+00,  6.6314e+00, -2.2974e+00]],  [[ 6.3961e+00,  6.7163e-02, -1.0543e-01]],  [[ 5.2773e+00,  1.6130e+00, -4.4109e-01]],  [[ 1.3451e+01, -1.9795e+00, -2.1000e+00]],  [[ 9.4836e+00,  4.7044e+00, -6.5308e-01]],  [[ 4.4989e+00,  2.2623e+00,  1.7637e+00]],  [[ 4.1425e+00,  3.5118e+00, -9.0276e-02]],  [[ 9.5909e+00,  7.2572e+00,

### Evaluation

In [ ]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 28.56
